In [1]:
import polars as pl
import os

In [2]:
juvenile_case_mapping = {
    'Juvenile-Felony': 'Felony',
    'Juvenile-Misdemeanor/Infraction': 'Misdemeanor',
    'Juvenile-Traffic Offense': 'Traffic Offense',
    
    'Juvenile-Status Offender': 'Status',
    
    'Juvenile-Abandoned': 'Dependency',
    'Juvenile-Abused': 'Dependency',
    'Juvenile-Neglected': 'Dependency',
    'Juvenile-Parental Jurisdiction': 'Dependency',
    'Juvenile-Paternity Determination': 'Dependency',
    'Juvenile-Guard Disruptd/Terminatd-P2013': 'Dependency',
    'Juvenile-Guardianship Disruptd/Terminatd': 'Dependency',
    'Juvenile-Homeless/Destitute/No Fault': 'Dependency',
    'Juvenile-Termination': 'Dependency',
    'Juvenile-Termination/Divorce-P2013': 'Dependency',
    'Juvenile-Voluntary Relinquishment': 'Dependency',
    'Juvenile-Voluntary Relinquishment-P2013': 'Dependency',

    # Cases that may be administrative but likely fall under Dependency
    'Juvenile-Interstate Compact-Holding Stat': 'Dependency',  
    'Juvenile-Migrated Cases - Douglas ONLY': 'Dependency'  
}


In [3]:
df = pl.read_database_uri("SELECT * FROM CaseInformation", os.environ["database"])

In [37]:
df.filter( pl.col("FiledDate") != "").with_columns(
    pl.col("FiledDate").map_elements(lambda x: x[-4:]).alias("Year").cast(pl.Int64)
).filter( pl.col("Year") > 2009)

CaseID,County,DocketNumber,DefendantName,DefendantType,Judge,Classification,FiledDate,TerminationDate,Year
str,str,str,str,str,str,str,str,str,i64
"""D 01 JV 20 000…","""Douglas""","""""","""Ariel Gordon""","""Juvenile""","""Amy Schuchman""","""Juvenile-Negle…","""09/25/2020""","""09/29/2020""",2020
"""D 01 JV 20 000…","""Douglas""","""""","""Kaelani Sison""","""Juvenile""","""Vernon Daniels…","""Juvenile-Negle…","""09/25/2020""","""12/07/2023""",2020
"""D 01 JV 20 000…","""Douglas""","""""","""Belina Inda""","""Juvenile""","""Candice Novak""","""Juvenile-Negle…","""09/25/2020""","""02/10/2022""",2020
"""D 01 JV 20 000…","""Douglas""","""""","""Cadance Saxon""","""Juvenile""","""Mary MZ Steven…","""Juvenile-Homel…","""09/23/2020""","""12/27/2023""",2020
"""D 01 JV 20 000…","""Douglas""","""""","""Sienna Volenti…","""Juvenile""","""Matthew R Kahl…","""Juvenile-Homel…","""09/23/2020""","""02/04/2021""",2020
"""D 01 JV 22 000…","""Douglas""","""""","""Sydney-Rae Cor…","""Juvenile""","""Amy Schuchman""","""Juvenile-Negle…","""11/22/2022""","""""",2022
"""D 01 JV 22 000…","""Douglas""","""""","""Ares Rodriguez…","""Juvenile""","""Vernon Daniels…","""Juvenile-Negle…","""11/21/2022""","""""",2022
"""D 01 JV 22 000…","""Douglas""","""""","""Kyro Wilson-Pu…","""Juvenile""","""Candice Novak""","""Juvenile-Negle…","""11/21/2022""","""03/21/2023""",2022
"""D 01 JV 22 000…","""Douglas""","""""","""Emmalina Escam…","""Juvenile""","""Amy Schuchman""","""Juvenile-Misde…","""11/18/2022""","""02/28/2024""",2022


In [4]:
df_bi = pl.read_database_uri("SELECT * FROM CaseBasicInfo", os.environ["database"])

In [ ]:
df_bi = df_bi.filter( pl.col("Type") == "JUV" )

In [19]:
df_bi = df_bi.with_columns(
    pl.col("ExactDOB").map_elements(lambda x: [d for d in x.replace("DOB: ", "").split("; ")])
).explode("ExactDOB").with_columns(
    pl.col("ExactDOB").map_elements(lambda x: x[-4:] ).alias("BY")
)

In [22]:
correct_bday = df_bi.filter( pl.col("BY") != "" ).with_columns(
    pl.col("BY").cast(pl.Int64)
).filter(pl.col("BY") == pl.col("YearOfBirth") )

In [28]:
correct_bday.filter(pl.col("BY") > 1992).group_by(["Person"]).count()

Person,count
str,u32
"""WIYUAL,WUOR""",1
"""CROOK,RHIANNA""",2
"""GAMBLE,KEIYARA…",1
"""PENN,ONYX,R""",2
"""RUCKER,CYTAYA""",1
"""GUSKO,YUKO,J""",4
"""NIETO,XENIAH""",2
"""WILLIAMS,JAVON…",1
"""NAGY,PATRICK""",1


In [20]:
no_bday = df_bi.filter( pl.col("BY") == "" )

In [ ]:
df_bi

In [21]:
no_bday

Person,Type,YearOfBirth,CaseID,ExactDOB,BY
str,str,i64,str,str,str
"""SISON,KAELANI""","""JUV""",2011,"""D 01 JV 20 000…","""""",""""""
"""SISON,NALA""","""JUV""",2013,"""D 01 JV 20 000…","""""",""""""
"""INDA,BELINA""","""JUV""",2014,"""D 01 JV 20 000…","""""",""""""
"""GALVAN,NICCOLO…","""JUV""",2011,"""D 01 JV 20 000…","""""",""""""
"""VOLENTINE,SIEN…","""JUV""",2006,"""D 01 JV 20 000…","""""",""""""
"""POPPINGO,STEVE…","""JUV""",1987,"""D 01 JV 10 903…","""""",""""""
"""LATHAN,DARRUS,…","""JUV""",2001,"""D 01 JV 10 903…","""""",""""""
"""JACKSON,ANTHON…","""JUV""",1969,"""D 01 JV 10 901…","""""",""""""
"""BLYTHE,CHAD,M""","""JUV""",1969,"""D 01 JV 10 901…","""""",""""""
